In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-data_test.pkl',
 'molecule-with-openbabel_test.pkl',
 'eachtype_te

In [5]:
mytrial = []

In [ ]:
"""
y=scalar_coupling_constant, type=0
257
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=1
454
{'columns': ['min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=2
681
{'columns': ['tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=3
139
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=4
219
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=5
299
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=6
379
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=7
449
{'columns': ['max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
"""

y=scalar_coupling_constant, type=0
257
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dis

In [6]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant']:
    for t in np.arange(3,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=scalar_coupling_constant, type=3
139
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_dis

In [9]:
for param in columns_list:
    print(len(param['columns']))

78
58
68
59
59
59
59
49


In [22]:
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 512)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':process_queue.param['columns'],
        'cv': {
            'cls': 'KFold',
            'init':{
                'n_splits': 5,
                'shuffle': True,
                'random_state': 42,
            },
        },
        'scaler': {
            'cls': 'StandardScaler',
            'init':{},
            'fit':{},
        },
        'model': {
            'cls': 'lgb.LGBMRegressor',
            'init': {
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':16
            },
            'fit': {
            },
        },
        'metric':'mean_absolute_error',
    }
    
    df_his, df_feature_importances, df_valid_pred, df_test_pred =  sk_process(df_train.sample(1000000), args, 'tune hyperparam', trial=mytrial, is_output_feature_importance=False, trial_level=0)
    val_metric_mean = np.mean(df_his.valid)
    return val_metric_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-07-08 12:43:35,887] Finished trial#0 resulted in value: 1.9479356066616114. Current best value is 1.9479356066616114 with parameters: {'learning_rate': 0.2557043291872395, 'feature_fraction': 0.8402360940571069, 'bagging_fraction': 0.6471727275908338, 'min_data_in_leaf': 762, 'lambda_l1': 0.001260157167891001, 'lambda_l2': 1.8109804288577593, 'max_bin': 49, 'num_leaves': 6, 'random_state': 8920}.
[I 2019-07-08 12:44:05,257] Finished trial#1 resulted in value: 1.0277087290901945. Current best value is 1.0277087290901945 with parameters: {'learning_rate': 0.467907656066654, 'feature_fraction': 0.9296597716002054, 'bagging_fraction': 0.88353122869927, 'min_data_in_leaf': 583, 'lambda_l1': 0.4844715238697431, 'lambda_l2': 8.647503479325444, 'max_bin': 78, 'num_leaves': 143, 'random_state': 2998}.
[I 2019-07-08 12:44:20,441] Finished trial#2 resulted in value: 1.5398453432881518. Current best value is 1.0277087290901945 with parameters: {'learning_rate': 0.467907656066654, 'feature_

[I 2019-07-08 13:01:35,485] Finished trial#38 resulted in value: 2.0455410292427043. Current best value is 0.9445406374765357 with parameters: {'learning_rate': 0.3003750010473155, 'feature_fraction': 0.7954806411666246, 'bagging_fraction': 0.825122687823091, 'min_data_in_leaf': 201, 'lambda_l1': 0.00010051426394331587, 'lambda_l2': 3.2239054643990496e-06, 'max_bin': 72, 'num_leaves': 234, 'random_state': 2988}.
[I 2019-07-08 13:02:07,136] Finished trial#39 resulted in value: 0.976087813882492. Current best value is 0.9445406374765357 with parameters: {'learning_rate': 0.3003750010473155, 'feature_fraction': 0.7954806411666246, 'bagging_fraction': 0.825122687823091, 'min_data_in_leaf': 201, 'lambda_l1': 0.00010051426394331587, 'lambda_l2': 3.2239054643990496e-06, 'max_bin': 72, 'num_leaves': 234, 'random_state': 2988}.
[I 2019-07-08 13:02:28,519] Finished trial#40 resulted in value: 1.0830722403332604. Current best value is 0.9445406374765357 with parameters: {'learning_rate': 0.300375

[I 2019-07-08 13:20:35,936] Finished trial#76 resulted in value: 0.9479237571940843. Current best value is 0.9445406374765357 with parameters: {'learning_rate': 0.3003750010473155, 'feature_fraction': 0.7954806411666246, 'bagging_fraction': 0.825122687823091, 'min_data_in_leaf': 201, 'lambda_l1': 0.00010051426394331587, 'lambda_l2': 3.2239054643990496e-06, 'max_bin': 72, 'num_leaves': 234, 'random_state': 2988}.
[I 2019-07-08 13:20:57,442] Finished trial#77 resulted in value: 1.244288848337931. Current best value is 0.9445406374765357 with parameters: {'learning_rate': 0.3003750010473155, 'feature_fraction': 0.7954806411666246, 'bagging_fraction': 0.825122687823091, 'min_data_in_leaf': 201, 'lambda_l1': 0.00010051426394331587, 'lambda_l2': 3.2239054643990496e-06, 'max_bin': 72, 'num_leaves': 234, 'random_state': 2988}.
[I 2019-07-08 13:21:31,993] Finished trial#78 resulted in value: 0.9763902372769845. Current best value is 0.9445406374765357 with parameters: {'learning_rate': 0.300375

[I 2019-07-08 13:39:40,905] Finished trial#114 resulted in value: 1.0492230333398693. Current best value is 0.9368553118374205 with parameters: {'learning_rate': 0.3166712257724028, 'feature_fraction': 0.9071277735212887, 'bagging_fraction': 0.7422147877975603, 'min_data_in_leaf': 416, 'lambda_l1': 1.5701746597674077e-06, 'lambda_l2': 1.7903507490427843e-05, 'max_bin': 98, 'num_leaves': 253, 'random_state': 6406}.
[I 2019-07-08 13:40:13,419] Finished trial#115 resulted in value: 0.9610546104863019. Current best value is 0.9368553118374205 with parameters: {'learning_rate': 0.3166712257724028, 'feature_fraction': 0.9071277735212887, 'bagging_fraction': 0.7422147877975603, 'min_data_in_leaf': 416, 'lambda_l1': 1.5701746597674077e-06, 'lambda_l2': 1.7903507490427843e-05, 'max_bin': 98, 'num_leaves': 253, 'random_state': 6406}.
[I 2019-07-08 13:40:40,681] Finished trial#116 resulted in value: 1.015126817371264. Current best value is 0.9368553118374205 with parameters: {'learning_rate': 0.3

[I 2019-07-08 13:59:47,959] Finished trial#152 resulted in value: 0.951212713884505. Current best value is 0.9368553118374205 with parameters: {'learning_rate': 0.3166712257724028, 'feature_fraction': 0.9071277735212887, 'bagging_fraction': 0.7422147877975603, 'min_data_in_leaf': 416, 'lambda_l1': 1.5701746597674077e-06, 'lambda_l2': 1.7903507490427843e-05, 'max_bin': 98, 'num_leaves': 253, 'random_state': 6406}.
[I 2019-07-08 14:00:26,436] Finished trial#153 resulted in value: 0.9329697749105812. Current best value is 0.9329697749105812 with parameters: {'learning_rate': 0.3414884085893545, 'feature_fraction': 0.9968942693539474, 'bagging_fraction': 0.6304650536121073, 'min_data_in_leaf': 391, 'lambda_l1': 1.0451097740407883e-06, 'lambda_l2': 0.039471193246930845, 'max_bin': 98, 'num_leaves': 256, 'random_state': 8004}.
[I 2019-07-08 14:01:03,998] Finished trial#154 resulted in value: 0.9415126520731034. Current best value is 0.9329697749105812 with parameters: {'learning_rate': 0.341

[I 2019-07-08 14:21:20,485] Finished trial#190 resulted in value: 1.0305378560799183. Current best value is 0.9300692164665992 with parameters: {'learning_rate': 0.29426859872688016, 'feature_fraction': 0.9991242137638949, 'bagging_fraction': 0.6352351289978297, 'min_data_in_leaf': 431, 'lambda_l1': 6.313437511621462e-06, 'lambda_l2': 0.016604298397879098, 'max_bin': 100, 'num_leaves': 255, 'random_state': 9214}.
[I 2019-07-08 14:21:52,453] Finished trial#191 resulted in value: 0.9697525930426156. Current best value is 0.9300692164665992 with parameters: {'learning_rate': 0.29426859872688016, 'feature_fraction': 0.9991242137638949, 'bagging_fraction': 0.6352351289978297, 'min_data_in_leaf': 431, 'lambda_l1': 6.313437511621462e-06, 'lambda_l2': 0.016604298397879098, 'max_bin': 100, 'num_leaves': 255, 'random_state': 9214}.
[I 2019-07-08 14:22:24,841] Finished trial#192 resulted in value: 0.9465400774373991. Current best value is 0.9300692164665992 with parameters: {'learning_rate': 0.29

In [23]:
df_trial = pd.DataFrame(mytrial)
df_trial[df_trial['message']=='tune hyperparam'][['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].sort_values(by=['val_metric_mean']).head()

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff
212,2019-07-08 14:06:29.634383,tune hyperparam,38,0.863236,0.930069,0.066833
215,2019-07-08 14:08:20.456876,tune hyperparam,38,0.862214,0.930458,0.068244
234,2019-07-08 14:18:47.324132,tune hyperparam,38,0.840667,0.932579,0.091912
202,2019-07-08 14:00:26.322634,tune hyperparam,38,0.855637,0.932970,0.077333
145,2019-07-08 13:30:42.870039,tune hyperparam,38,0.866630,0.936855,0.070225


In [11]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [12]:
columns_list = [{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [13]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [18]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-16 11:18:23.406847,modeling for 0,78,0.820160,0.933006,0.112845,modeling for 0
1,2019-07-16 11:18:40.952761,modeling for 3,59,0.161547,0.200190,0.038643,modeling for 3
2,2019-07-16 11:18:43.612953,modeling for 1,58,0.286269,0.433112,0.146842,modeling for 1
3,2019-07-16 11:18:50.776218,modeling for 4,59,0.154898,0.237684,0.082786,modeling for 4
4,2019-07-16 11:19:50.783660,modeling for 2,68,0.360945,0.391322,0.030378,modeling for 2
5,2019-07-16 11:20:19.215675,modeling for 6,59,0.196629,0.225852,0.029223,modeling for 6
6,2019-07-16 11:21:38.304023,modeling for 5,59,0.427043,0.453386,0.026343,modeling for 5
7,2019-07-16 11:21:46.386701,modeling for 7,49,0.102380,0.141882,0.039502,modeling for 7


In [16]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.44499493261917566
0 0.9330056960257216
3 0.2001904951609826
1 0.4331118692460489
4 0.23768410461908276
2 0.39132241115217037
6 0.22585247088385893
5 0.4533857586621615
7 0.14188187674382496


In [22]:
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.210720,0.871862,0.746697,1.206203,1.007102
2505538,7163685,3.809454,3.867529,2.588954,2.918856,3.701579
2505539,7163686,3.658385,3.428395,3.517420,1.978553,3.407225
2505540,7163687,4.435589,4.727233,5.057265,4.668085,4.264797
2505541,7163688,118.513458,118.589493,116.857854,117.165564,112.934275


In [23]:
idx=0.44
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)